# Introduction
Here we investigate the theoretical maximum pathway yield of two different butanone production pathways.

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction
import numpy as np
from matplotlib import pyplot as plt

In [2]:
pth = cobra.io.read_sbml_model('../../model/p-thermo.xml')

### Anaerobic production of compounds of interest
We saw in the GSMM paper of Mol et al., (2021, https://doi.org/10.1016/j.ymben.2021.03.002) that there are some metabolites the model cannot synthesize anaerobically, though experimental data proves they can. As we do not have enough information to fix this through adding correct pathways in the model, instead we will add these components to the model medium. This also puts the requirement of needing exhcanges and transports for these compounds that I will code in.

As we do not have information about the way these metabolites are transported across the membrane, i will add them as passive transport. 

In [3]:
#optimize for growth with a little bit of oxygen
#to check objective later is the same
pth.reactions.EX_o2_e.bounds = (-0.1,1000)
pth.optimize().objective_value

0.12613059342838315

In [4]:
#list of metabolites that need to be taken up
mets=['btn','thmpp','sprm','spmd','adocbl','10fthf','fe3','pheme']

In [5]:
for met in mets:
    #create extracellular metabolite
    emet = Metabolite(id=f"{met}_e", 
                      name = f"extracellular {met}", 
                      formula = pth.metabolites.get_by_id( f"{met}_c").formula, #copy formula & charge from intracell met
                      charge = pth.metabolites.get_by_id( f"{met}_c").charge, 
                      compartment ='e')
    pth.add_metabolites(emet)
    #create transport reaction
    rct = Reaction(id=f"{met.upper()}t", name = f"{met} transport")
    #add intra and extra cellular metabolites
    rct.add_metabolites({
        pth.metabolites.get_by_id( f"{met}_c"):-1,
        pth.metabolites.get_by_id( f"{met}_e"):1
    })
    #define transport bounds as irreversible
    rct.bounds = (-1000,1000)
    #Add transport reaction to the model
    pth.add_reaction(rct)
    #create exchange for the extracellular metabolite
    pth.add_boundary(pth.metabolites.get_by_id(f"{met}_e"),
                     type='exchange',
                     reaction_id = f"EX_{met}_e",
                     lb = -1, ub= 1000) #set lb to -1 mmol/gcdw/h to have some excess but not endless amounts

In [6]:
#check the model can grow anaerobically
pth.reactions.EX_o2_e.bounds=(0,1000)

In [7]:
pth.optimize().objective_value

0.12435312325992394

So we have now inserted the acetone pathway, as well as KO'ed the LDH reaction. we slso supplied the different cofactors and allowed their uptake. I will then save this model so we can use it in furhter dFBA. 

In [8]:
cobra.io.write_sbml_model(pth,"p-thermo_anaerobic.xml")

## Butanone through 2,3-BDO
Find max theoretical butanone production when going through 2,3-butanediol, as shown in DOI: 10.1002/cssc.201300853 and doi:10.1371/journal.pone.0102774

In [9]:
model = cobra.io.read_sbml_model('p-thermo_anaerobic.xml')

In [10]:
#add the butanone metabolite, both intracellular & extracellular
model.add_metabolites(Metabolite(id='buto_c',formula = 'C4H8O', name = 'Butanone', compartment = 'c' ))

The conversion to butanediol is already present in the GSMM. I'll only add the conversion to butanone & export and then find the max yield from pyruvate.

In [11]:
#Add DH reaction
DH_but = Reaction("DH")
DH_but.name = "Diol dehydratase - butanone forming"

DH_but.add_metabolites({
    model.metabolites.buto_c: 1.0,      #butanone
    model.metabolites.rr23bdo_c: -1.0,   #butanediol
    model.metabolites.h2o_c: 1.0,     #water
    
})
print(DH_but.check_mass_balance())    #ensure correct mass balance
print(DH_but.bounds)                  #Ensure correct bounds
print(DH_but.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
rr23bdo_c --> buto_c + h2o_c


In [12]:
#add all the reactions to the model
model.add_reactions([DH_but])

In [13]:
#aad butanone sink
model.add_boundary(model.metabolites.get_by_id("buto_c"), type="sink")

Reaction identifier,SK_buto_c
Name,Butanone sink
Memory address,0x01bf8391ad90
Stoichiometry,buto_c <=> Butanone <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [14]:
#aad pyr feed
model.add_boundary(model.metabolites.get_by_id("pyr_c"), type="sink")

Reaction identifier,SK_pyr_c
Name,Pyruvate sink
Memory address,0x01bf83644c70
Stoichiometry,pyr_c <=> Pyruvate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [15]:
#set pyr to 10 mmol/gcdw/h
model.reactions.SK_pyr_c.bounds = (-10,1000)

In [16]:
#aerobic conditions
model.reactions.EX_o2_e.bounds =  (-1000,1000)

In [17]:
#no glucose
model.reactions.EX_glc__D_e.bounds = (0,0)

In [18]:
#optimize butanediol production, assuming no biomass production
model.objective = model.reactions.SK_buto_c

In [19]:
#turn off hydrogen uptake
model.reactions.EX_h_e.bounds = (0,1000)

In [20]:
#max yield
print(model.optimize().objective_value/10)

0.3466452961672476


In [21]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h2o_e,EX_h2o_e,3.866,0,0.00%
o2_e,EX_o2_e,0.9345,0,0.00%
pyr_c,SK_pyr_c,10,3,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-6.134,1,20.45%
for_e,EX_for_e,-10,1,33.33%
buto_c,SK_buto_c,-3.466,4,46.22%


In [22]:
#try anaerobically
model.reactions.EX_o2_e.bounds =  (0,1000)

In [23]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h2o_e,EX_h2o_e,4.211,0,0.00%
pyr_c,SK_pyr_c,10,3,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-3.674,2,24.49%
co2_e,EX_co2_e,-5.789,1,19.30%
for_e,EX_for_e,-6.326,1,21.09%
buto_c,SK_buto_c,-2.634,4,35.12%


In [24]:
#max yield
print(model.optimize().objective_value/10)

0.26343561643835633


### ketovaleryl coA pathway
The second pathway we investigate if from http://dx.doi.org/10.1128/AEM.03964-15, using propionyl-coA (ppcoa_c) as starting metabolite, which can be produced from pyruvate in the model already. 

In [25]:
model = cobra.io.read_sbml_model('p-thermo_anaerobic.xml')

In [26]:
#add the butanone metabolite, both intracellular & extracellular
model.add_metabolites(Metabolite(id='buto_c',formula = 'C4H8O', name = 'Butanone', compartment = 'c' ))

In [27]:
#add 3-ketovaleryl-CoA
model.add_metabolites(Metabolite(id='kvcoa_c',formula = 'C26H38N7O18P3S', charge = -4, name = '3-ketovaleryl-CoA', compartment = 'c' ))

In [28]:
#add 3-oxopentanoate
model.add_metabolites(Metabolite(id='oxop_c',formula = 'C5HO3',charge = -1, name = '3-oxopentanoate', compartment = 'c' ))

In [29]:
#Add reaction to 3-ketovaleryl
PHA_B = Reaction("PHA_B")
PHA_B.name = "3-ketovaleryl forming"

PHA_B.add_metabolites({
    model.metabolites.ppcoa_c: -1.0,      #propionyl coa
    model.metabolites.accoa_c: -1.0,   #acetyl coa
    model.metabolites.coa_c: 1.0,     #coa
    model.metabolites.kvcoa_c: 1.0,     #3-ketovalerylcoa
    model.metabolites.h_c: -3
    
})
print(PHA_B.check_mass_balance())    #ensure correct mass balance
print(PHA_B.bounds)                  #Ensure correct bounds
print(PHA_B.reaction)                #Ensure correct reaction

{'charge': -4.0, 'H': -4.0}
(0.0, 1000.0)
accoa_c + 3 h_c + ppcoa_c --> coa_c + kvcoa_c


In [30]:
#Add reaction to 3-oxopentanoate_with acetate
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "3-oxopentanoate forming"

ctfAB_ac.add_metabolites({
    model.metabolites.kvcoa_c: -1.0,      #3-ketovalerylcoa
    model.metabolites.oxop_c: 1.0,   #3-oxopentanoate
    model.metabolites.ac_c: -1.0,     #acetate
    model.metabolites.accoa_c: 1.0,     #acetylcoa
    
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{'H': -6.0}
(0.0, 1000.0)
ac_c + kvcoa_c --> accoa_c + oxop_c


In [31]:
#Add reaction to 3-oxopentanoate_with propionate
ctfAB_pr = Reaction("ctfAB_pr")
ctfAB_pr.name = "3-oxopentanoate forming"

ctfAB_pr.add_metabolites({
    model.metabolites.kvcoa_c: -1.0,      #3-ketovalerylcoa
    model.metabolites.oxop_c: 1.0,   #3-oxopentanoate
    model.metabolites.prpnte_c: -1.0,     #propionate
    model.metabolites.ppcoa_c: 1.0,     #propionylcoa,
    model.metabolites.h_c: -1
    
})
print(ctfAB_pr.check_mass_balance())    #ensure correct mass balance
print(ctfAB_pr.bounds)                  #Ensure correct bounds
print(ctfAB_pr.reaction)                #Ensure correct reaction

{'H': -6.0}
(0.0, 1000.0)
h_c + kvcoa_c + prpnte_c --> oxop_c + ppcoa_c


In [32]:
#Add reaction to butanone
ABC_but = Reaction("ABC_but")
ABC_but.name = "butanone forming"

ABC_but.add_metabolites({
    model.metabolites.oxop_c: -1.0,      #3-oxopentanoate
    model.metabolites.buto_c: 1.0,   #butanone
    model.metabolites.co2_c: 1.0, #CO2
    model.metabolites.h_c: -1
    
})
print(ABC_but.check_mass_balance())    #ensure correct mass balance
print(ABC_but.bounds)                  #Ensure correct bounds
print(ABC_but.reaction)                #Ensure correct reaction

{'H': 6.0}
(0.0, 1000.0)
h_c + oxop_c --> buto_c + co2_c


Note there is a charge discrepancy here, however, we can observe that the two reactions taken together have a net charge balance and so this won't skew the predictions too much for this purpose.

In [33]:
#add all the reactions to the model
model.add_reactions([PHA_B, ctfAB_ac,ctfAB_pr,ABC_but])

In [34]:
#aad butanone sink
model.add_boundary(model.metabolites.get_by_id("buto_c"), type="sink")

Reaction identifier,SK_buto_c
Name,Butanone sink
Memory address,0x01bf84301ca0
Stoichiometry,buto_c <=> Butanone <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [35]:
#aad pyr feed
model.add_boundary(model.metabolites.get_by_id("pyr_c"), type="sink")

Reaction identifier,SK_pyr_c
Name,Pyruvate sink
Memory address,0x01bf84301820
Stoichiometry,pyr_c <=> Pyruvate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [36]:
#set pyr to 10 mmol/gcdw/h
model.reactions.SK_pyr_c.bounds = (-10,1000)

In [37]:
#aerobic conditions
model.reactions.EX_o2_e.bounds =  (-1000,1000)

In [38]:
#no glucose
model.reactions.EX_glc__D_e.bounds = (0,0)

In [39]:
#optimize butanediol production, assuming no biomass production
model.objective = model.reactions.SK_buto_c

In [40]:
#turn of hydrogen uptake
model.reactions.EX_h_e.bounds = (0,1000)

In [41]:
#max yield
print(model.optimize().objective_value/10)

0.16666666666666669


In [42]:
#try anaerobically
model.reactions.EX_o2_e.bounds =  (0,1000)

In [43]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
h2o_e,EX_h2o_e,7.924,0,0.00%
pyr_c,SK_pyr_c,10,3,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-5.682,2,37.88%
co2_e,EX_co2_e,-2.076,1,6.92%
for_e,EX_for_e,-10.44,1,34.80%
buto_c,SK_buto_c,-1.53,4,20.40%


In [44]:
#max yield
print(model.optimize().objective_value/10)

0.15302895086321391
